# Visualize network

The purpose of this notebook is to help visualize the metabolic network via Escher and ensure that the map and model are synchronized with each other.

## Using Escher online:
Brief steps on how to load the RBC metabolic network map using Escher online. 
1. Go to https://escher.github.io/#/ and set both the map and model options to *None*. 
2. Click the *Load map* button to open a blank canvas.
3. Use *(Ctrl+M or Cmd+M)* to load a COBRA model from a `json` file. Alternatively, click on the *Model* tab, then the *Load COBRA model JSON* option to load a new model.
    * Load the model file **"RBC-GEM.json"** from the `/model` directory.
4. Use *(Ctrl+O or Cmd+O)* to load an Escher map from a `json` file. Alternatively, click on the *Map* tab, then the *Load map JSON* option to load a new map.
    * Load the map file **"RBC-GEM.full.map.json"** from the `/map` directory.

## Using Escher via python API (Not recommended currently):
It is currently not recommend to utilize the python API as the current Escher dependencies conflict with recent versions of jupyter. It is therefore up to the user to manage package dependencies to utilize the Python API for Escher. 

The best way to do this is to utilize a seperate virtual environment into prevent dependency conflicts and install Escher. 
1. Install Python 3.9
2. Run the following lines to install packages (order matters!):

    ```
    cd /code # Navigate to code directory where the pyproject.toml file is located.
    pip install escher 
    ```

## Additional information
See the [documentation for Escher](https://escher.readthedocs.io/en/latest/) for additional details on how to use Escher.


King ZA, Dräger A, Ebrahim A, Sonnenschein N, Lewis NE, Palsson BO. Escher: A Web Application for Building, Sharing, and Embedding Data-Rich Visualizations of Biological Pathways. PLoS Comput Biol. 2015 Aug 27;11(8):e1004321. doi: 10.1371/journal.pcbi.1004321. PMID: 26313928; PMCID: PMC4552468.
## Setup
### Import packages

In [1]:
import datetime
import json
from pathlib import Path

import matplotlib as mpl
import pandas as pd
from rbc_gem_utils import (
    GEM_NAME,
    MAP_NAMES,
    ensure_iterable,
    get_dirpath,
    read_cobra_model,
    show_versions,
)

show_versions()

/var/folders/5t/hk8m3g6d1jn25x5rssjgsrmm0000gn/T/ipykernel_77579/255983133.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd



Package Information
-------------------
rbc-gem-utils 0.0.1

Dependency Information
----------------------
beautifulsoup4                       4.12.3
bio                                   1.6.2
cobra                                0.29.0
depinfo                               2.2.0
kaleido                               0.2.1
matplotlib                            3.8.2
memote                               0.17.0
networkx                              3.2.1
notebook                              6.5.6
openpyxl                              3.1.2
pandas                                2.2.0
pre-commit                            3.6.0
pyvis                                 0.3.2
rbc-gem-utils[database,network,vis] missing
requests                             2.31.0
scipy                                1.12.0
seaborn                              0.13.2

Build Tools Information
-----------------------
pip        23.3.1
setuptools 68.2.2
wheel      0.41.2

Platform Information
-------------------

## Load RBC-GEM model
* Load the XML model to utilize annotations in any data mapping and visualization (guarunteed to have annotations).
* Use the JSON model to check against the model file that gets loaded into Escher.

In [2]:
model_dirpath = get_dirpath("model")
model = read_cobra_model(filename=model_dirpath / f"{GEM_NAME}.xml")
model

Name,RBC_GEM
Memory address,7fc4a13225b0
Number of metabolites,2157
Number of reactions,3281
Number of genes,820
Number of groups,78
Objective expression,1.0*NaKt - 1.0*NaKt_reverse_db47e
Compartments,"cytosol, extracellular space"


## Load map JSON

In [3]:
save_figures = True

print(MAP_NAMES)
map_name = "RBC-GEM.full.map"
log_level = "OFF"

map_json_filepath = get_dirpath("map") / f"{map_name}.json"
results_path = get_dirpath(use_temp="processed") / model.id / "Maps"
results_path.mkdir(exist_ok=True, parents=True)

{'RBC-GEM.full.map'}


### Format data for viewing on map

In [4]:
try:
    import escher

    escher.rc["never_ask_before_quit"] = True
except ImportError:
    escher = None

In [5]:
reaction_data = {}
metabolite_data = {}
gene_data = {}
reaction_scale = []
metabolite_scale = []

#### Example: categorized by subsystems

In [6]:
df_pathways = pd.read_csv(
    get_dirpath("curation") / "subsystems.tsv", sep="	", dtype=str
)
df_pathways["category"] = df_pathways["category"].replace(
    "Metabolism of other amino acids", "Amino acid metabolism"
)

categories_to_exclude = {"Pseudoreactions", "Model total"}

cmax = 0.8
colors = {
    "Amino acid metabolism": mpl.colors.to_hex(mpl.cm.spring(cmax)),
    "Carbohydrate metabolism": mpl.colors.to_hex(mpl.cm.Greens(cmax)),
    "Lipid metabolism": mpl.colors.to_hex(mpl.cm.Blues(cmax)),
    "Metabolism of cofactors and vitamins": mpl.colors.to_hex(mpl.cm.summer(cmax)),
    "Nucleotide metabolism": mpl.colors.to_hex(mpl.cm.winter(cmax)),
    "Reactive species": mpl.colors.to_hex(mpl.cm.Reds(cmax)),
    "Transport reactions": mpl.colors.to_hex(mpl.cm.Purples(cmax)),
    "Other": mpl.colors.to_hex(mpl.cm.gray_r(cmax)),
}

reaction_scales_mapping = {
    subsystem: {"type": "value", "value": f"{val}", "color": f"{color}"}
    for val, (subsystem, color) in enumerate(colors.items(), start=1)
}


reaction_data = {}
df_cat_subsystems = df_pathways.groupby("category")["name"].agg(lambda x: list(x))
for category, subsystem_list in df_cat_subsystems.items():
    if category in categories_to_exclude:
        continue
    if (
        category not in reaction_scales_mapping
        and category not in categories_to_exclude
    ):
        category = "Other"

    reaction_data.update(
        {
            reaction.id: reaction_scales_mapping[category]["value"]
            for group in model.groups.get_by_any(subsystem_list)
            for reaction in group.members
        }
    )

reaction_scale = list(reaction_scales_mapping.values())
if escher is not None:
    builder = escher.Builder(
        map_json=map_json_filepath,
        # model=model,
        model_json=get_dirpath(use_temp="model") /{GEM_NAME}.json",
    )

    for attr, value in dict(
        reaction_data=reaction_data,
        metabolite_data=metabolite_data,
        gene_data=gene_data,
        reaction_scale=reaction_scale,
        metabolite_scale=metabolite_scale,
    ).items():
        if value:
            setattr(builder, attr, value)
    if save_figures:
        builder.save_html(
            results_path / "html" / f"{GEM_NAME}_categorized_subsystems.html"
        )

### Export data for web browser

In [32]:
# val = 1
# color = "blue"
# reaction_data = {
#     'FADRy': 1,
#     'FLVRy': 1,
#     'FADL': 1,
#     'FMNAT': 1,
#     'CFMNH': 1,
#     'FMNRy': 1,
#     'FMNPH': 1,
#     'RBFK': 1
# }


# mets = set()
# genes = set()

# for met, name in {
#     'rblvrd_c': 'Reduced riboflavin',
#     'fadh2_c': 'FADH2',
#     'nadph_c': 'NADPH',
#     'hb_c': 'Deoxyhemoglobin',
#     'nadp_c': 'NADP+',
#     'amp_c': 'AMP',
#     'ribflv_e': 'Riboflavin',
#     'h_c': 'H+',
#     'fmnh2_c': 'FMNH2',
#     'h2o_c': 'H2O',
#     'methb_c': 'Methemoglobin',
#     'ribflv_c': 'Riboflavin',
#     '45cfmn_c': "Riboflavin cyclic-4',5'-phosphate",
#     'ppi_c': 'Diphosphate',
#     'pi_c': 'Orthophosphate',
#     'adp_c': 'ADP',
#     'fmn_c': 'FMN',
#     'fad_c': 'FAD',
#     'atp_c': 'ATP',
# }.items():
#     met = model.metabolites.get_by_id(met)
#     met.name = name
# for r in model.reactions.get_by_any([
#     'FADRy', 'FMNRy', 'FLVRy', 'FADL', 'FMNAT', 'CFMNH', 'FMNPH', 'RBFK', "RIBFLVABCte", "RIBFLVt", "METHBFMNR"
# ]):
#     mets.update(r.metabolites)
#     genes.update(r.genes)

# for m in mets:
#     print(f"'{m.id}': '{m.name}',")

# # for g in genes:
# #     print(g.annotation.get("hgnc.symbol"))

# if escher is not None:
#     builder = escher.Builder(
#         map_json="/Users/zhaiman/Desktop/Riboflavin.json",
#         model=model,
#     )
#     builder.save_html(
#         "~/Desktop/Riboflavin.html",
#     )

In [ ]:
with open(
    get_dirpath(use_temp="interim") / "reaction_map_data.json", "w"
) as map_datafile:
    json.dump(reaction_data, map_datafile)

with open(
    get_dirpath(use_temp="interim") / "metabolite_map_data.json", "w"
) as map_datafile:
    json.dump(metabolite_data, map_datafile)

## Ensure model and map are synchronized

In [ ]:
with open(map_json_filepath) as mapfile:
    map_json = json.load(mapfile)
    print(f"Loaded map as JSON object: {map_name}\n")

metadata_json = map_json[0]
print(pd.Series(metadata_json))
model_json = map_json[1]

### Check reactions
Use this code to find which reactions are missing from the map. Helpful for identifying any ID conversions or new additions that need to be made.

In [ ]:
include_pseudoreactions = False
include_transports = False
map_rxns = {
    reaction_dict["bigg_id"] for reaction_dict in model_json["reactions"].values()
}
model_reactions = model.reactions
if not include_pseudoreactions:
    model_reactions = model_reactions.query(lambda x: not x.boundary)

if not include_transports:
    model_reactions = model_reactions.query(lambda x: len(x.compartments) == 1)

model_reactions = set(model_reactions.list_attr("id"))


previous_id_mapping_df = pd.read_csv(
    get_dirpath("deprecatedIdentifiers") / "reactions_deprecatedIdentifiers.tsv",
    sep="\t",
    index_col=None,
)

previous_id_mapping_df["rxnRetired"] = previous_id_mapping_df["rxnRetired"].str.split(
    ";"
)
previous_id_mapping_df = previous_id_mapping_df.explode("rxnRetired")
rxns_retired_id_replacements = previous_id_mapping_df[
    previous_id_mapping_df["rxnRetired"].isin(map_rxns)
]
rxns_retired_id_replacements = rxns_retired_id_replacements.set_index("rxnRetired")[
    "rxns"
].to_dict()


found_in_map = model_reactions.intersection(map_rxns)
missing_rxns_from_map = model_reactions.difference(map_rxns).difference(
    rxns_retired_id_replacements
)

print(f"Number of biochemical reactions in model: {len(model_reactions)}")
if include_transports:
    print(
        f"Number of transport reactions in map: {len(model_reactions.query(lambda x: len(x.compartments) != 1))}"
    )
print(
    f"Number of reactions not in map due to ID retirement: {len(rxns_retired_id_replacements)}\n"
)
print(
    f"Number of reactions not in map for alternative reasons: {len(missing_rxns_from_map)}\n"
)
missing_rxns_from_map

### Check metabolites
Use this code to find which metabolites are missing from the map. Helpful for identifying any ID conversions or new additions that need to be made.

In [ ]:
compartments = ["c"]

map_mets = {
    node_dict["bigg_id"]
    for node_dict in model_json["nodes"].values()
    if node_dict["node_type"] == "metabolite"
}


previous_id_mapping_df = pd.read_csv(
    get_dirpath("deprecatedIdentifiers") / "metabolites_deprecatedIdentifiers.tsv",
    sep="\t",
    index_col=None,
)

previous_id_mapping_df["metRetired"] = previous_id_mapping_df["metRetired"].str.split(
    ";"
)
previous_id_mapping_df = previous_id_mapping_df.explode("metRetired")
mets_retired_id_replacements = previous_id_mapping_df[
    previous_id_mapping_df["metRetired"].isin(map_mets)
]
mets_retired_id_replacements = mets_retired_id_replacements.set_index("metRetired")[
    "mets"
].to_dict()

# missing_from_model = map_mets.difference(model_mets) # Not always accurate due to
if not compartments:
    compartments = model.compartments
compartments = ensure_iterable(compartments)

for comp in compartments:
    model_mets = set(
        model.metabolites.query(lambda x: x.compartment == comp).list_attr("id")
    )
    found_in_map = model_mets.intersection(map_mets)
    missing_mets_from_map = model_mets.difference(map_mets).difference(
        mets_retired_id_replacements.values()
    )

    print(
        f"Number of metabolites in model ({model.compartments[comp]}): {len(model_mets)}"
    )
    print(
        f"Number of metabolites in map ({model.compartments[comp]}): {len(found_in_map)}"
    )
    print(
        f"Number of metabolites not in map due to ID retirement ({model.compartments[comp]}): {len(mets_retired_id_replacements)}\n"
    )
    print(
        f"Number of metabolites not in map for alternative reasons ({model.compartments[comp]}): {len(missing_mets_from_map)}\n"
    )


missing_mets_from_map

### Update Map
#### Metadata

In [ ]:
metadata_json["map_name"] = map_name
metadata_json["map_description"] = datetime.datetime.strftime(
    datetime.date.today(), "\nLast Modified %a %b %d %Y"
)
print(pd.Series(metadata_json))

#### Map identifier replacement

In [ ]:
rxns_id_replacements = {
    # Old ID on Map: New ID for Map
}
rxns_id_replacements.update(rxns_retired_id_replacements)
mets_id_replacements = {
    # Old ID on Map: New ID for Map
}
mets_id_replacements.update(mets_retired_id_replacements)
# Update identifiers
for reaction_dict in model_json["reactions"].values():
    reaction_dict["bigg_id"] = rxns_id_replacements.get(
        reaction_dict["bigg_id"], reaction_dict["bigg_id"]
    )
    for node_dict in reaction_dict["metabolites"]:
        node_dict["bigg_id"] = mets_id_replacements.get(
            node_dict["bigg_id"], node_dict["bigg_id"]
        )

for node_dict in model_json["nodes"].values():
    if node_dict["node_type"] != "metabolite":
        continue
    node_dict["bigg_id"] = mets_id_replacements.get(
        node_dict["bigg_id"], node_dict["bigg_id"]
    )

In [ ]:
with open(map_json_filepath, "w") as mapfile:
    json.dump(obj=[metadata_json, model_json], fp=mapfile)
    print(f"Saved map as JSON object: {map_name}\n")

### Utilize EscherConverter to convert map to a standard format
To run the Escher converter, uncomment out the final cell
* GitHub Page: https://github.com/draeger-lab/EscherConverter
* Instructions for the Escher converter: https://escher.readthedocs.io/en/stable/escherconverter.html

In [ ]:
convert_to = "Escher"
# convert_to = None

if convert_to is not None:
    converter_path = get_dirpath(use_temp="external") / "EscherConverter-1.2.1.jar"
    converted_output_filepath = {
        "SBGN": get_dirpath("map") / f"{map_name}.sbgn",
        "SBML": get_dirpath("map") / f"{map_name}.xml",
    }[convert_to]
    !java -jar -Xms8G -Xmx8G -Duser.language=en "$converter_path" --input="$map_json_filepath" --output="$converted_output_filepath" --gui=false --log-level="$log_level"